In [ ]:
from selenium import webdriver # pip install selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time as tm
from datetime import date 

In [ ]:
# ! pip install python-dotenv
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

ACCOUNT_SID = os.environ.get("ACCOUNT_SID")
AUTH_TOKEN = os.environ.get("AUTH_TOKEN")
SENDER_WHATSAPP_NUMBER = os.environ.get("SENDER_WHATSAPP_NUMBER")
RECIPIENT_WHATSAPP_NUMBER = os.environ.get("RECIPIENT_WHATSAPP_NUMBER")

In [ ]:
# selenium 4
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager # pip install webdriver-manager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [ ]:
import time
import random

def search_ticket(search_string):
    driver.get(search_string)
    wait = WebDriverWait(driver, 10)

    # xpath_string = """//*[@id="16operatorfilter"]/span[1]""" 
    xpath_string = r"/html/body/div[7]/div[2]/div[5]/div/div[1]/div[2]/div/label[1]/span[1]" # Full xpath. Uncomment if doesn't work
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpath_string)))
    element.click()
    # print("SR Clicked")

    # xpath_string = """//*[@id="1typesfilter"]/span[1]"""
    xpath_string = """/html/body/div[7]/div[2]/div[5]/div/div[1]/div[3]/div/label[1]/span[1]""" # Full xpath. Uncomment if doesn't work
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpath_string)))
    element.click()
    # print("AC clicked")

    elements = driver.find_elements(By.CSS_SELECTOR, "[data-title='Seats Available']")
    # Uncomment below if above generates too many values
    # elements = driver.find_elements(By.CSS_SELECTOR,"td.tbl_col5[data-title='Seats Available']")

    # Iterate through the elements and print their text content
    num_of_tickets = []
    for element in elements:
        try:
            value = int(element.text.replace("\n", "").strip())
            num_of_tickets.append(value)
        except ValueError:
            # Value is NOT integer
            continue
    
    return num_of_tickets

from twilio.rest import Client # pip install twilio
# Function to send a WhatsApp message
def send_whatsapp_message(message):
    client = Client(ACCOUNT_SID, AUTH_TOKEN)
    message = client.messages.create(
        body=message,
        from_=SENDER_WHATSAPP_NUMBER,
        to=RECIPIENT_WHATSAPP_NUMBER
    )
    print(f"WhatsApp message sent. SID: {message.sid}")

SEARCH_STRING = r"https://www.shohoz.com/booking/bus/search?fromcity=Bogura&tocity=Dhaka&doj=02-Jul-2023&dor="
while True:
    current_time = time.strftime('%Y-%m-%d %H:%M:%S')  # Get the current time
    num_of_tickets = search_ticket(SEARCH_STRING)
    print(f"{current_time}\t Tickets found {num_of_tickets}")
    if any(num_of_tickets):
        break
    interval = random.randint(2, 5) * 60
    time.sleep(interval)

import winsound

def generate_alarm():
    # Define the frequency and duration of the alarm sound
    frequency = 2500  # In hertz (Hz)
    duration = 10000  # In milliseconds (ms)
    on_time = 1000
    for _ in range(duration//on_time):
        winsound.Beep(frequency, on_time)
        time.sleep(1)

while True:
    msg = f"Wake up!!!\nTicket found: {num_of_tickets}.\nMore info: {SEARCH_STRING}"
    send_whatsapp_message(msg)
    generate_alarm()
